
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0)                                |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |   Changes the session type to Glue ETL.                                                                                                                   |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer                       |

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
# Retrieve the list of existing buckets

import boto3

s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

In [ ]:
## Read data from Amazon S3 and create DataFrame

## Replace the {S3_PATH} below, with your bucket name.

s3_path = "{S3_PATH}" 

df = spark.read.load("s3://" + s3_path + "/input/lab2/sample.csv", 
                          format="csv", 
                          sep=",", 
                          inferSchema="true",
                          header="true")

## print schema
df.printSchema()

## show 10 records

df.show(10)


In [ ]:
## Write data as Parquert 

df.write.parquet("s3://" + s3_path + "/input/lab2/output/parquet/")

In [ ]:
## reading parquet file

dfpaquet = spark.read.parquet("s3://" + s3_path + "/input/lab2/output/parquet/")

In [ ]:
# spark, sparkDFcsv are from the previous example
# Print the schema in a tree format

dfpaquet.printSchema()


In [ ]:
## print 5 records

dfpaquet.show(5)

In [ ]:
## count the total no. of rows in DataFrame

dfpaquet.count()


In [ ]:
# Select only the "Country" column
dfpaquet.select('Country')


In [ ]:
# Select only the "Country" column
# calling action

dfpaquet.select('Country').show()


In [ ]:
dfpaquet.select('Country').show(10,truncate=False)


In [ ]:
### show multiple columns and create new dataframe

dfselect = dfpaquet.select(dfpaquet['Country'], dfpaquet['ItemType'], dfpaquet['SalesChannel'],dfpaquet['TotalRevenue'])

dfselect.show(10,truncate=False)


In [ ]:
### Filter by country

dfselect.filter(dfpaquet['Country'] == 'United Kingdom').show(10,truncate=False)


In [ ]:
### Filter by country and Total Revenue and create new dataframe

dfselectfilter = dfselect.filter((dfpaquet['Country'] == 'United Kingdom') & (dfpaquet['TotalRevenue'] <= 200000.00))

dfselectfilter.show(10,truncate=False)


In [ ]:
## perform GroupBy operation 

dfselectfiltergroupby = dfselectfilter.groupBy("ItemType").sum("TotalRevenue")

dfselectfiltergroupby.show(10,truncate=False)


In [ ]:
## perform Order By Operation 

dfselectfiltergroupbyorderby = dfselectfiltergroupby.orderBy("sum(TotalRevenue)", ascending=False)

dfselectfiltergroupbyorderby.show(10,truncate=False)


## Spark createOrReplaceTempView()

How does the createOrReplaceTempView() method work in Spark and what is it used for? One of the main advantages of Apache Spark is working with SQL along with DataFrame/Dataset API. So if you are comfortable with SQL, you can create a temporary view on DataFrame/Dataset by using createOrReplaceTempView() and using SQL to select and manipulate the data.

A Temporary view in Spark is similar to a real SQL table that contains rows and columns but the view is not materialized into files. In this article, we will be discussing what is createOrReplaceTempView() and how to use it to create a temporary view and run Spark SQL queries.

In [ ]:
## converting datafrem to createOrReplaceTempView table

dfpaquet.createOrReplaceTempView('dfpaquetsql')

In [ ]:
spark.sql('select * from dfpaquetsql limit 10').show()

In [ ]:
spark.sql('select Country, ItemType , SalesChannel , TotalRevenue from dfpaquetsql limit 10').show()

In [ ]:
spark.sql('select ItemType , sum(TotalRevenue) as SumTotalRevenue from dfpaquetsql group by ItemType order by SumTotalRevenue asc limit 10').show()

In [ ]:
dfcsvsave = spark.sql('select ItemType , sum(TotalRevenue) as SumTotalRevenue from dfpaquetsql group by ItemType order by SumTotalRevenue asc')

In [ ]:
## Save as CSV and creating a single file
## coalese is very expensive action, it's not recommeneded to use with large dataset

singlefile = dfcsvsave.coalesce(1) 

singlefile.write.format("csv").mode('overwrite').save("s3://" + s3_path + "/input/lab2/output/csv/")

In [ ]:
## stop the current session 

%stop_session